In [1]:
import numpy as np
import pandas as pd
import random
import re
import matplotlib as plt

In [70]:
a=pd.read_excel(r'C:\Users\49653\Desktop\yx\data\data.xlsx')  ##读取数据集
a

,事故编号,事故形态,事故地点,事故发生日期
0,310101420190004101,11,长乐路成都南路北约50米,2019/9/1 0:45:00
1,310101420190004100,11,西藏南路淮海东路北约6米,2019/9/1 4:44:00
2,310101420190004153,11,陕西南路出进贤路南约10米,2019/9/1 10:18:00
3,310101420190004121,11,斜土东路出制造局路东约60米,2019/9/1 11:09:00
4,310101420190004123,36,白渡路中华路东南约25米,2019/9/1 11:45:00
...,...,...,...,...
194,310101420190004599,11,斜土路进蒙自路东约100米,2019/9/30 15:23:00
195,310101420190004600,11,牛庄路云南北路东约3米,2019/9/30 21:15:00
196,310101420190004696,11,光启南路进复兴东路南约50米,2019/9/30 22:37:00
197,310101420190004607,35,六合路宁波路西南约15米,2019/10/1 15:30:00


In [71]:
##将数据进行一个拆分，转换为数值化数据

didian=['0','1']  ##0为公路，1为隧道
fangxiang1=['0','北','南','西','东','东北','东南','西北','西南'] ##北 南 西 东 北西 南西 北东 南东
ri=['0','1']  ## 1-15上旬  16以下下旬
shike=['0','1']  ##早6：00-17：59 为0，晚为18：00-05：59 为1
label=[11,12,19,21,22,23,29,31,32,33,34,35,36,37,38,39]  ## 事故形态,为特征
## 示例长乐路成都南路北约50米 2019/9/1 11
## 1 50 1 0 0 11 
shiguxingtai=a['事故形态']
shigudidian=a['事故地点']
date=a['事故发生日期']

In [77]:
def qiantao(zifu,zifuchuan):  ##字符串匹配特定字符位置
    return [substr.start() for substr in re.finditer(zifu, zifuchuan)]
def didianleibie(didianleibie):  ## 判断所属地点
    addr = qiantao('隧道',didianleibie)
    if(addr ==[]):
        return 1
    else:
        return 0
def juli(juli1):  ##判断所属距离，缺失的后续会补上
    addr =qiantao('约',juli1)
    if(addr==[]):  ## 代表没有距离
        return 0
    else:
        addr1=re.sub("\D", "", juli1)  ##仅保留数字
        if(addr1[0]=='0' and len(addr1)>1):
            return addr1[1]
        else:
            return addr1
    
def panduanri(time):  ##判断所属日
    addr1=qiantao('/',time)[1] ##取第二个/
    addr2=qiantao(' ',time)[0]  ##取空格 
    if(int(time[addr1+1:addr2])<15):
        return 0
    else:
        return 1
def panduanshi(time1):
    addr1=qiantao(' ',time1)[0]  ##取空格
    addr2=qiantao(':',time1)[0]  ##取英文冒号
    if(6<int(time1[addr1+1:addr2])<18):
        return 1
    else:
        return 0   
def fangxiang(fang):
    try:
        if(didianleibie(fang)):  ##对路的进行判断
            b1=qiantao('路',fang)[-1]
            b2=qiantao('约',fang)[0]
            jie=qiantao('街',fang[b1:])
            qiao=qiantao('桥',fang[b1:])
            if(jie!=[]):
                return fangxiang1.index(fang[b1+jie[-1]+1:b2])
            elif(qiao!=[]):
                return fangxiang1.index(fang[b1+qiao[-1]+1:b2])
            else:
                if(len(fang[b1+1:b2])>2):
                    return fangxiang1.index(fang[b1+1:b2][-1])
                else:
                    return fangxiang1.index(fang[b1+1:b2])
        else:
            return 0
    except:  ##特殊数据提取分析
        if(qiantao('约',fang)!=[]):
            b1=qiantao('街',fang)[-1]
            b2=qiantao('约',fang)[0]
            return fangxiang1.index(fang[b1+1:b2])
        else:
            return fangxiang1.index(fang[-1])

In [78]:
type1=[]
length1=[]
forward1=[]
ri1=[]
shi1=[]
label1=a['事故形态']
for i in range(len(shigudidian)):
    type1.append(didianleibie(shigudidian[i]))
    length1.append(juli(shigudidian[i]))
    forward1.append(fangxiang(shigudidian[i]))
    ri1.append(panduanri(date[i]))
    shi1.append(panduanshi(date[i]))

In [79]:
c=pd.DataFrame(columns=['type','length','forward','ri','shi','label'])

In [80]:
c['type']=type1
c['length']=length1
c['forward']=forward1
c['ri']=ri1
c['shi']=shi1
c['label']=label1

In [81]:
c.to_csv('newdata.csv',encoding='utf-8')

In [47]:
## test
for i in range(len(shigudidian)):
    print("原数据为",shigudidian[i],date[i])
    print(str(didianleibie(shigudidian[i]))+" "+str(juli(shigudidian[i]))+" "+str(panduanri(date[i]))+" "+str(panduanshi(date[i]))+" "+str(fangxiang(shigudidian[i]))+" "+str(shiguxingtai[i]))
    print(i)

原数据为 长乐路成都南路北约50米 2019/9/1 0:45:00
1
1 50 0 0 1 11
0
原数据为 西藏南路淮海东路北约6米 2019/9/1 4:44:00
1
1 6 0 0 1 11
1
原数据为 陕西南路出进贤路南约10米 2019/9/1 10:18:00
1
1 10 0 0 2 11
2
原数据为 斜土东路出制造局路东约60米 2019/9/1 11:09:00
1
1 60 0 0 4 11
3
原数据为 白渡路中华路东南约25米 2019/9/1 11:45:00
1
1 25 0 0 6 36
4
原数据为 复兴中路进思南路东约30米 2019/9/1 21:30:00
1
1 30 0 1 4 35
5
原数据为 九江路进西藏中路西约80米 2019/9/2 19:16:00
2
1 80 0 1 3 11
6
原数据为 延安东路黄陂北路南约01米 2019/9/3 2:47:00
3
1 1 0 0 2 11
7
原数据为 瞿溪路出局门路东约30米 2019/9/3 7:30:00
3
1 30 0 0 4 11
8
原数据为 温州路新闸路北约0米 2019/9/3 8:42:00
3
1 0 0 0 1 21
9
原数据为 丽园路鲁班路东约0米 2019/9/3 10:07:00
3
1 0 0 0 4 11
10
原数据为 鲁班路南塘浜路南约10米 2019/9/3 10:30:00
3
1 10 0 0 2 11
11
原数据为 浙江中路进牛庄路北约15米 2019/9/3 11:10:00
3
1 15 0 0 1 11
12
原数据为 人民路进方浜中路北约50米 2019/9/3 13:07:00
3
1 50 0 0 1 11
13
原数据为 南昌路进思南路东约100米 2019/9/3 13:45:00
3
1 100 0 0 4 11
14
原数据为 徐家汇路肇周路西约15米 2019/9/3 19:40:00
3
1 15 0 1 3 11
15
原数据为 鲁班路斜土路东约0米 2019/9/4 0:10:00
4
1 0 0 0 4 11
16
原数据为 光启南路进复兴东路南约50米 2019/9/4 3:30:00
4
1 50 0 0 2 21
17
原数据为 黄陂北路进江阴路北约10米 2019/9/

In [84]:
## 事故高发上下旬，早晚，
a

,Unnamed: 0,type,length,forward,ri,shi,label
0,0,1,50,1,0,0,11
1,1,1,6,1,0,0,11
2,2,1,10,2,0,1,11
3,3,1,60,4,0,1,11
4,4,1,25,6,0,1,36
...,...,...,...,...,...,...,...
194,194,1,100,4,1,1,11
195,195,1,3,4,1,0,11
196,196,1,50,2,1,0,11
197,197,1,15,8,0,1,35


147

8
48
35
38
55
2
3
4
6


'北'

In [34]:
b

157

,事故编号,事故形态,事故地点,事故发生日期
0,310101420190004101,11,长乐路成都南路北约50米,2019/9/1 0:45:00
1,310101420190004100,11,西藏南路淮海东路北约6米,2019/9/1 4:44:00
2,310101420190004153,11,陕西南路出进贤路南约10米,2019/9/1 10:18:00
3,310101420190004121,11,斜土东路出制造局路东约60米,2019/9/1 11:09:00
4,310101420190004123,36,白渡路中华路东南约25米,2019/9/1 11:45:00
...,...,...,...,...
194,310101420190004599,11,斜土路进蒙自路东约100米,2019/9/30 15:23:00
195,310101420190004600,11,牛庄路云南北路东约3米,2019/9/30 21:15:00
196,310101420190004696,11,光启南路进复兴东路南约50米,2019/9/30 22:37:00
197,310101420190004607,35,六合路宁波路西南约15米,2019/10/1 15:30:00


In [62]:
a11

['2019/9/1 0:45:00',
 '2019/9/1 4:44:00',
 '2019/9/1 10:18:00',
 '2019/9/1 11:09:00',
 '2019/9/1 11:45:00',
 '2019/9/1 21:30:00',
 '2019/9/2 19:16:00',
 '2019/9/3 2:47:00',
 '2019/9/3 7:30:00',
 '2019/9/3 8:42:00',
 '2019/9/3 10:07:00',
 '2019/9/3 10:30:00',
 '2019/9/3 11:10:00',
 '2019/9/3 13:07:00',
 '2019/9/3 13:45:00',
 '2019/9/3 19:40:00',
 '2019/9/4 0:10:00',
 '2019/9/4 3:30:00',
 '2019/9/4 7:40:00',
 '2019/9/4 7:59:00',
 '2019/9/4 8:00:00',
 '2019/9/4 21:30:00',
 '2019/9/5 8:50:00',
 '2019/9/5 9:29:00',
 '2019/9/5 19:18:00',
 '2019/9/6 7:35:00',
 '2019/9/6 7:57:00',
 '2019/9/6 8:20:00',
 '2019/9/6 9:34:00',
 '2019/9/6 11:11:00',
 '2019/9/7 7:41:00',
 '2019/9/7 9:53:00',
 '2019/9/7 11:21:00',
 '2019/9/7 14:30:00',
 '2019/9/7 17:53:00',
 '2019/9/7 22:07:00',
 '2019/9/8 10:19:00',
 '2019/9/8 13:30:00',
 '2019/9/8 13:40:00',
 '2019/9/8 20:29:00',
 '2019/9/9 13:45:00',
 '2019/9/9 14:45:00',
 '2019/9/9 15:54:00',
 '2019/9/9 21:40:00',
 '2019/9/10 13:20:00',
 '2019/9/10 13:30:00',
 '20

8
1
9
6
3
5
6
4
4
5
9
10
9
3
2
4
13
7
11
4
11
10
7
12
10
6
7
5
3
5


In [66]:
print(len(ri))

199
